# Plato Tutorial

Plato is a package made with two objectives.

1) Simplify the use of Theano.  
2) Provide a good libary of standard Deep Learning algorithms.

This tutorial demonstrates point 1.  It's useful but not necessary to have a basic knowledge of theano to do this tutorial.

## 1. Symbolic functions.

In Plato, we have the concept of *symbolic functions*, which are function that take and return theano symbolic variables.  These functions can be compiled to *numeric functions* which take and return numpy arrays and python ints/floats.

In [1]:
from plato.core import symbolic

@symbolic
def add_two_numbers(x, y):
    return x+y

f = add_two_numbers.compile()
print '3+4=%s' % f(3, 4)

3+4=7


Internally, here is what happens: On the first (and in this case, only) call to add_two_numbers, Plato gets the inputs (3, 4), looks at their type (both scalar integers in this case), and compiles a symbolic expression that adds them together.  The equivalent code using just theano would be:

In [1]:
import theano
from theano.tensor import scalar

x = scalar(dtype = 'int32')
y = scalar(dtype = 'int32')
z = x+y

f = theano.function(inputs = [x, y], outputs = z)
print '3+4=%s' % f(3, 4)


3+4=7


## 2. Adding State

We are also able to create stateful functions.  In the below example, Plato recognises that the return value is in the format `output, [(shared_variable, shared_variable_update)]`, so it creates a stateful function when it compiles.

In [2]:
import theano
from plato.core import symbolic

@symbolic
def counter(start_value = 1):
    counter = theano.shared(start_value)
    return counter, [(counter, counter+1)]

f = counter.compile()
print 'I can count to ten.  See: %s' % ([int(f()) for _ in xrange(10)])

# Note that we start from scratch when we compile the function a new time, 
# because the shared variable is initialized within the function call
f2 = counter.compile()
print 'I can too: %s' % ([int(f2()) for _ in xrange(10)])


I can count to ten.  See: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
I can too: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


# 3. Classes: Multiple functions sharing a variable.

We often have situations where we have a variable that is shared between two functions (e.g. in a classifier, the weights may be modified by the *train* function and used by the *predict* function).  We usually deal with this using classes.  As a simple example, lets take the [Collatz Conjecture](https://en.wikipedia.org/wiki/Collatz_conjecture).  

In [3]:
class Collatz:

    def __init__(self, initial_value):
        self.value = theano.shared(initial_value)
        
    @symbolic
    def divide_by_2(self):
        new_value = self.value/2
        return new_value, [(self.value, new_value)]
    
    @symbolic
    def multiply_by_3_and_add_one(self):
        new_value = self.value*3+1
        return new_value, [(self.value, new_value)]
    
c = Collatz(20)
div_fcn = c.divide_by_2.compile()
mul_fcn = c.multiply_by_3_and_add_one.compile()

value = c.value.get_value()
print 'Demonstrate Collatz conjecture for initial value of %s' % c.value.get_value()

while value != 1:
    value = div_fcn() if value % 2 == 0 else mul_fcn()
    print value

print 'Note that since the value is attached to the class, it persists if functons are recompiled.'
new_div_fcn = c.divide_by_2.compile()
new_mul_fcn = c.multiply_by_3_and_add_one.compile()
for _ in xrange(6):
    value = new_div_fcn() if value % 2 == 0 else new_mul_fcn()
    print value

Demonstrate collatz conjecture for initial value of 20
10
5
16
8
4
2
1
Note that since the value is attached to the class, it persists if functons are recompiled.
4
2
1
4
2
1


## 3. Callable Classes

In python, classes can also act as functions, if they implement a `__call__` method.  This can be useful when you want to make parameterized functions.  Therefore Plato also allows you to decorate callable classes.  For example:

In [3]:
from plato.core import symbolic

@symbolic
class MultiplyBySomething:
    
    def __init__(self, what):
        self.what = what
        
    def __call__(self, x):
        return x*self.what
    
f = MultiplyBySomething(3).compile()

print '3*4=%s' % f(4)

3*4=12


## 4. Debugging

A big advantage of Plato is easier debugging.  There are two ways in which Plato helps you debug.

### 4A: Test values

Theano allows you to add "test-values" to your symbolic variables ([see tutorial](http://deeplearning.net/software/theano/tutorial/debug_faq.html)).  This helps to catch shape-errors at compile-time instead of run-time, where it is difficult to find the line of code that caused them.  However, it can be a bit of extra work for the programmer, because they have to manually attach test values to their variables.  Fortunately, since Plato compiles your functions on the first pass, it can attach test-values "under the hood".

For example, lets look at a matrix multiplication, where we accidently get the shapes of our matrices wrong.  Since all inputs are given test values, we can easily track down the error - the traceback will lead back to the correct line.  This would not have been possible without test values, because the error would occur in the compiled code, which is no-longer linked to the source code.

In [2]:
import numpy as np
from plato.core import symbolic


@symbolic
def forward_pass(x, w):
    print 'x-shape: %s' % (x.tag.test_value.shape, )
    print 'w-shape: %s' % (w.tag.test_value.shape, )
    # Note that the above test-values only display on the first iteration.
    return x.dot(w)

f = forward_pass.compile()

try:
    # The following will cause an error (because second argument should have shape (4, 3))
    h = f(np.random.randn(5, 4), np.random.rand(3, 4))  
except ValueError as err:
    # If you do not catch the error, you get a stacktrace which points to the line at fault.
    print '%s: %s' % (err.__class__.__name__, err.message)


x-shape: (5, 4)
w-shape: (3, 4)
ValueError: shapes (5,4) and (3,4) not aligned: 4 (dim 1) != 3 (dim 0)


### 4B: Variable Traces

It can also be useful to print/plot traces of intermediate values.  Ordinarily in theano, this would require setting those variables as outputs, and restructuring code to peek at what would normally be an internal variables.  Plato does a bit of magic which allows you to print/plot/do anything with internal variables.  The following example illustrates this: 

In [5]:
import numpy as np
from plato.core import symbolic, tdbprint
import theano.tensor as tt
import theano

class Layer:
    
    def __init__(self, w):
        self.w = theano.shared(w)
        
    @symbolic
    def forward_pass(self, x):
        pre_sigmoid = x.dot(self.w)
        tdbprint(pre_sigmoid, name = 'Pre-Sigmoid Activation')
        y = tt.nnet.sigmoid(pre_sigmoid)
        return y
    
n_samples = 1
n_in = 4
n_out = 3
    
layer = Layer(np.random.randn(n_in, n_out))
fwd_fcn = layer.forward_pass.compile()
y = fwd_fcn(np.random.randn(n_samples, n_in))
print 'Post Sigmoid Activation: %s' % (y)

Pre-Sigmoid Activation: [[-2.36296192 -0.45285114  0.94697635]]
Post Sigmoid Activation: [[ 0.08604099  0.3886831   0.72050669]]


## 5. Enforcing Interfaces

In larger programs, it can be useful to enforce interfaces - that is, functions are required to obey a certain contract.  This allows function A to use function B without knowing in particular which function it is.  For instance, you may have some code that iterates through a dataset and trains a predictor, but doesn't necessarily know what kind of predictor it is - just that it has a *train* function that accepts inputs and targets, and returns updates.

For this reason, we have an extended set of decorators which enforce type-checking on inputs/outputs.  Currently all these decorators just specfy outputs, but this can easily be extended to to type-checking for inputs too.

`symbolic_stateless` - Returns a single output variable.  
`symbolic_updater` - Returns a list of variable updates: [(shared_var_0, new_val_0), (shared_var_1, new_val_1), ...]  
`symbolic_standard` - Returns a tuple of outputs and a list of updates.  ((out_0, ...), [(shared_var_0, new_val_0), ...])  

If you decorate with these, you get a `SymbolicFormatError` when your inputs/outputs are not in the expected format.  For example:

In [7]:
from plato.core import symbolic_standard, SymbolicFormatError

@symbolic_standard
def multiply_by_two(x):
    y = 2*x
    return y  # Bad! We decorated with "standard" but did not return the standard format of (outputs, updates)

f = multiply_by_two.compile()

print 'Trying to run improperly formatted function...'
try: 
    f(3)
except SymbolicFormatError as err:
    print '  %s: %s' % (err.__class__.__name__, err.message)

print 'Lets try again with the correct format....'

@symbolic_standard
def multiply_by_two_in_correct_format(x):
    y = 2*x
    return (y, ), []  # !!! Correct format.

f_again = multiply_by_two_in_correct_format.compile()

print '  [3*2] = %s' % (f_again(3), )

Trying to run improperly formatted function...
  SymbolicFormatError: You did not return a 2-tuple of outputs, updates.  You returned Elemwise{mul,no_inplace}.0
Lets try again with the correct format....
  [3*2] = [array(6)]


## 6. Converting Formats

In the previous example, we saw that the `multiply_by_two` function was much cleaner than the `multiply_by_two_in_correct_format` function.  But if were to pass this function into some generic code that wants to treat functions interchangeably, we needed to implement it in the correct format.  To overcome this impracticality of forcing users to either implement functions in a messy way, or wrap them, symbolic functions have a **to_format** method to do format-conversion.  

As an example, lets imaging you're making a generic "Chain" function.  Chain's job is to compose a list of (possible stateful) functions into a chain: $f_c = f_1 \circ f_2 \circ ... f_n$

In our example, we'll have two functions - Multiply by two, and a cumulative sum.

Note a few things here:  
- We don't know, when creating Chain, what formats the inner funtions will have (will they return updates, will they return a tuple of outputs or just a single one)
- We therefore don't know what format the output of Chain should take (should it return just one output or a tuple, should it return updates, etc)

The below example shows how we can use the **to_format** method to convert a symbolic function defined in one format to another.

In [23]:
from plato.core import symbolic, symbolic_standard, symbolic_stateless
import theano
import numpy as np

@symbolic_standard
class Chain:
    
    def __init__(self, *functions):
        self.functions = functions
        
    def __call__(self, *args):
        updates = []
        for f in self.functions:
            # Note how we convert to the standard format to get results in the (outputs, updates) format.
            args, new_updates = f.to_format(symbolic_standard)(*args)  # "*" is python syntax for unpacking a tuple as arguments.
            updates += new_updates
        return args, updates

@symbolic_stateless
def multiply_by_two(x):
    return x*2

@symbolic
def running_sum(x):
    z = theano.shared(0)
    new_z = z+x
    return new_z, [(z, new_z)]

f = Chain(multiply_by_two, running_sum).compile()

for i in xrange(10):
    print 'sum([0..%s]*2) = %s ' % (i, f(i))


sum([0..0]*2) = [array(0)] 
sum([0..1]*2) = [array(2)] 
sum([0..2]*2) = [array(6)] 
sum([0..3]*2) = [array(12)] 
sum([0..4]*2) = [array(20)] 
sum([0..5]*2) = [array(30)] 
sum([0..6]*2) = [array(42)] 
sum([0..7]*2) = [array(56)] 
sum([0..8]*2) = [array(72)] 
sum([0..9]*2) = [array(90)] 


## 7. Fixed Arguments

When you use a numpy array on a theano symbolic function, it treats it as a constant.  We can use the **fixed_args** argument to **compile()** to partially-specify a function.  For example:


In [27]:
from plato.core import symbolic

@symbolic
def multiply(x, y):
    return x*y

f_mult_by_3 = multiply.compile(fixed_args = dict(y=3))

print '2*3 = %s' % f_mult_by_3(2)
print '5*3 = %s' % f_mult_by_3(5)


2*3 = 6
5*3 = 15
